# __Homework 4:__ Practical analysis with BioPython

For the homework, you are going to extend the code from the analysis of our FASTQ file in lectures 8 and 9.
Recall that the FASTQ file contains reads from a real sequencing run of influenza virus HA and NA genes.

---
The __actual sequences__ are as follows:

    5'-[end of HA]-AGGCGGCCGC-[16 X N barcode]-3'
or 

    5'-[end of NA]-AGGCGGCCGC-[16 X N barcode]-3'
---


__The end of NA is__ `...CACGATAGATAAATAATAGTGCACCAT`
    
__The end of HA is__ `...CCGGATTTGCATATAATGATGCACCAT`

---    

    
The __sequencing reads__ from the reverse end of the molecules (in 5'>3' orientation), so the sequencing reads are as follows:

    5'-[reverse complement of 16 X N barcode]-GCGGCCGCCT-[reverse complement of the end of HA]-3'
or

    5'-[reverse complement of 16 X N barcode]-GCGGCCGCCT-[reverse complement of the end of NA]-3'

---   
    
The reads can originate from **either** HA or NA, and that will be distinguished by the most 3' end of the read.
But in our example exercise in class, we did not distinguish among reads matching to HA and NA, as we didn't even look far enough into the read to tell the identity.

For the homework, your goal is to write code that extends the material from lectures 8 and 9 to also distinguish between HA and NA.
This homework can be completed almost entirely by re-using code from lecture 9. You will need to set up your analysis to do the following:
 1. Get the reverse complement of each read.
 2. Determine if it matches the expected pattern for HA and NA, and if so which one.
 3. If it matches, extract the barcode and add it to a dictionary to keep track of counts.
 4. Determine the number and distribution of barcodes for HA and NA separately.

Please include code to address each of the following questions. Please include code comments to explain what your code is attempting to accomplish. Don't forget to include references to the sources you used to obtain your answer, including your classmates (if you are working in groups).  

1. How many reads map to HA, and how many reads map to NA?

In [74]:
# load necessary packages
import re
import Bio.SeqIO
import Bio.Seq

# read sequences from the barcodes_R1.fastq file and create a list of only the sequences (as Seq objects)
# used code from lecture09.ipynb practice 
seqreads = list(Bio.SeqIO.parse('barcodes_R1.fastq', 'fastq'))
seqreads_Seq = []
for seqrecord in seqreads:
    seqreads_Seq.append(seqrecord.seq)

# go through each sequence, find the reverse complement, and create lists of 
# Seq objects that belong to either NA or HA sequences (depending on the end sequences)
NA_seqs = []
HA_seqs = []
neither = []
end_NA = 'CACGATAGATAAATAATAGTGCACCAT'
end_HA = 'CCGGATTTGCATATAATGATGCACCAT'

for seq in seqreads_Seq:
    rev_comp = seq.reverse_complement()
    if end_NA in rev_comp:
        NA_seqs.append(rev_comp)
    elif end_HA in rev_comp:
        HA_seqs.append(rev_comp)
    else:
        neither.append(rev_comp)

# print out the number of reads mapping to NA or HA
print('There are ' + str(len(HA_seqs)) + ' reads that map to HA.')
print('There are ' + str(len(NA_seqs)) + ' reads that map to NA.')

There are 5409 reads that map to HA.
There are 4122 reads that map to NA.


2. How many HA sequences did not have a valid barcode? Also anwer the same question for NA.

In [75]:
# create variables for the lenth of the barcode (bclen) and the sequence upstream of the barcode (upstream)
bclen=16
upstream='AGGCGGCCGC'

# create dictionaries (one for NA, one for HA) in which the keys correspond to the barcodes in the sequence and 
# the values correspond to the number of times in which that barcode appears

# used code from lecture09.ipynb

NA_barcode_counts = {'nonvalid':0}
for seq in NA_seqs:
    str_seq = str(seq)
    NA_barcode_search = re.compile(end_NA + upstream + '(?P<NA_barcode>[ATCG]{' + str(bclen) + '})')
    NA_match = NA_barcode_search.search(str_seq)   
    if NA_match:
        NA_barcode_match = NA_match.group('NA_barcode') 
        if NA_barcode_match in NA_barcode_counts:
            NA_barcode_counts[NA_barcode_match] += 1
        else:
            NA_barcode_counts[NA_barcode_match] = 1
    else:
        NA_barcode_counts['nonvalid'] += 1

HA_barcode_counts = {'nonvalid':0}
for seq in HA_seqs:
    str_seq = str(seq)
    HA_barcode_search = re.compile(end_HA + upstream + '(?P<HA_barcode>[ATCG]{' + str(bclen) + '})')
    HA_match = HA_barcode_search.search(str_seq)   
    if HA_match:
        HA_barcode_match = HA_match.group('HA_barcode') 
        if HA_barcode_match in HA_barcode_counts:
            HA_barcode_counts[HA_barcode_match] += 1
        else:
            HA_barcode_counts[HA_barcode_match] = 1
    else:
        HA_barcode_counts['nonvalid'] += 1

# print out the number of sequences that do not have a valid barcode for NA and HA
print('There are ' + str(HA_barcode_counts['nonvalid']) + ' HA sequences that do not have a valid barcode.')
print('There are ' + str(NA_barcode_counts['nonvalid']) + ' NA sequences that do not have a valid barcode.')

There are 160 HA sequences that do not have a valid barcode.
There are 213 NA sequences that do not have a valid barcode.


3. What is the HA barcode with the most counts (and how many counts)? Also answer the same question for NA.

    _Hint: you will need to find the key associated with the maximum value in your dictionary. There are many ways to do this._

In [76]:
# used code from previous undergrad Python class

####### HA

# remove 'nonvalid' key from dictionary
if 'nonvalid' in HA_barcode_counts:
    del HA_barcode_counts['nonvalid']

# sort the counts from the dictionary and extract the highest count
HA_count_list = list(HA_barcode_counts.values())
HA_count_list.sort(reverse=True)
highest_HA_count = HA_count_list[0]

# find the barcode with the highest count
for (barcode, count) in HA_barcode_counts.items():
    if count == highest_HA_count:
        barcode_highest_HA_count = barcode

print('The HA barcode with the most counts is ' + str(barcode_highest_HA_count) + ' with ' + str(highest_HA_count) + ' counts.')

####### NA

# remove 'nonvalid' key from dictionary
if 'nonvalid' in NA_barcode_counts:
    del NA_barcode_counts['nonvalid']

# sort the counts from the dictionary and extract the highest count
NA_count_list = list(NA_barcode_counts.values())
NA_count_list.sort(reverse=True)
highest_NA_count = NA_count_list[0]

# find the barcode with the highest count
for (barcode, count) in NA_barcode_counts.items():
    if count == highest_NA_count:
        barcode_highest_NA_count = barcode

print('The NA barcode with the most counts is ' + str(barcode_highest_NA_count) + ' with ' + str(highest_NA_count) + ' counts.')

The HA barcode with the most counts is CCCGACCCGACATTAA with 155 counts.
The NA barcode with the most counts is ACCAGTTCTCCCCGGG with 152 counts.
